In [4]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path, latest_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [5]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}!")

Using device: cuda!


In [6]:
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = latest_weights_file_path(config)
print(f"Loading weights from {model_filename}")
# state = torch.load("weights/tmodel_39.pt", map_location=torch.device('cpu'))
state = torch.load("weights/betsi.pt", map_location=torch.device('cpu'))
model.load_state_dict(state['model_state_dict'])

Max length of source sentence: 309
Max length of target sentence: 274
weights_files: [PosixPath('weights/betsi.pt')]
Loading weights from weights/betsi.pt


<All keys matched successfully>

In [7]:
def load_next_batch():
    batch = next(iter(val_dataloader))
    encoder_input = batch['encoder_input'].to(device)
    encoder_mask = batch['encoder_mask'].to(device)
    decoder_input = batch['decoder_input'].to(device)
    decoder_mask = batch['decoder_mask'].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(i) for i in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(i) for i in decoder_input[0].cpu().numpy()]

    model_out = greedy_decode(model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config["seq_len"], device)
    model_out_text = vocab_tgt.decode(model_out.detach().cpu().numpy())

    return batch, encoder_input_tokens, decoder_input_tokens, model_out_text

In [8]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame([(r, c, float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [9]:
batch, encoder_input_tokens, decoder_input_tokens, output = load_next_batch()
sentence_len = encoder_input_tokens.index("[PAD]")
print(f"Source: {batch['src_text'][0]}")
print(f"Target: {batch['tgt_text'][0]}")
print(f"Output: {output}")
print(f"{sentence_len = }")

Source: In fact I notice a quite unjustifiable hostility on the part of the public toward these men, and it seems to me that it is envy...'
Target: Ma, al contrario, c’è un certo atteggiamento ostile, basato su nulla, della società verso queste persone, e mi sembra che ci sia dell’invidia....
Output: Ma , al contrario , c ’ è un certo atteggiamento ostile , su di questi rapporti con le quali mi sembra , e mi pare che per me sia un ’ invidia ...
sentence_len = 30


In [10]:
layers = [i for i in range(3)]
heads = [i for i in range(8)]

get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [11]:
# Decoder self attention

get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [12]:
# Cross attention

get_all_attention_maps("encoder-decoder", layers, heads, encoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)